## Crawling 

In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
from lib import competition_to_family_map, get_all_battles, url_leaders_by_weight, extract_all_profiles_weight, SQLiteConnection

In [3]:
import datetime

In [4]:
# abrir Google Chrome
driver = webdriver.Chrome(r"C:\Users\manuel.lopez\Documents\chromedriver.exe")

### Extract all battles from profile

In [13]:
from_date = datetime.datetime.strptime('1 Nov 2018', '%d %b %Y')

In [14]:
battles = get_all_battles(driver, '3238', from_date)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 10.0.14393 x86_64)


In [8]:
battles

,wins,local,opponent,event,date,round,local_points,opponent_points,duration,category,has_video,competition_family,url_video
0,0,LOPEZ SHERIFF Estrella,PARK Da Sol,Grand Slam Osaka 2018,2018-11-23,Round 1,0,100,00:12,-52 kg,True,Grand Slam,https://judobase.ijf.org/#/competition/contest...
1,0,LOPEZ SHERIFF Estrella,PUPP Reka,Grand Prix The Hague 2018,2018-11-16,Round 2,0,10,04:00,-52 kg,True,Grand Prix,https://judobase.ijf.org/#/competition/contest...
2,1,LOPEZ SHERIFF Estrella,JIMENEZ Kristine,Grand Prix The Hague 2018,2018-11-16,Round 1,10,0,04:00,-52 kg,True,Grand Prix,https://judobase.ijf.org/#/competition/contest...


### Extract top competitors from every weight

In [5]:
competitors_all = list()

for weight in url_leaders_by_weight.keys():
    print(weight, '...')
    competitors = extract_all_profiles_weight(driver, weight)
    competitors_all.append(competitors)

men_60 ...
men_66 ...
men_73 ...
men_81 ...
men_90 ...
men_100 ...
men_100+ ...
women_48 ...
women_52 ...
women_57 ...
women_63 ...
women_70 ...
women_78 ...
women_78+ ...


In [11]:
competitors_all = pd.concat(competitors_all)

In [14]:
competitors_all.sample(5)

,name,country,profile_id,category
56,WEZEU DOMBEU Helene,CMR,4393,women_63
83,GONZALEZ Oritia,ARG,2675,women_52
58,NJIE Faye,GAM,27553,men_73
21,VARAPAYEU Yahor,BLR,13091,men_90
19,PARK Yujin,KOR,15228,women_78


#### Save to db

In [16]:
conn = SQLiteConnection('../judo.sql')

In [ ]:
conn.add_table('competitors', competitors_all, if_exists='fail')

### Extract recent contests from a list of competitors

In [35]:
for index, profile_id in enumerate(competitors.profile_id):
    print(index)
    battles_girl, petes = get_all_battles(driver, girl_id)

    battles_all.append(battles_girl)
    petes_all.append(petes)

0
Extracting info from: ABE Hifumi
Extracting video URLs
1
Extracting info from: MARGVELASHVILI Vazha
Extracting video URLs
2
Extracting info from: FLICKER Tal
Extracting video URLs
3
Extracting info from: AN Baul
Extracting video URLs
4
Extracting info from: SHMAILOV Baruch
Extracting video URLs
5
Extracting info from: ZANTARAIA Georgii
Extracting video URLs
6
Extracting info from: GANBOLD Kherlen
Extracting video URLs
7
Extracting info from: GOMBOC Adrian
Extracting video URLs
8
Extracting info from: ZHUMAKANOV Yeldos
Extracting video URLs
9
Extracting info from: CARGNIN Daniel
Extracting video URLs
10
Extracting info from: SERIKZHANOV Yerlan
Extracting video URLs
11
Extracting info from: PULIAEV Mikhail
Extracting video URLs
12
Extracting info from: SHAMILOV Yakub
Extracting video URLs
13
Extracting info from: MARUYAMA Joshiro
Extracting video URLs
14
Extracting info from: VIERU Denis
Extracting video URLs
15
Extracting info from: CHIBANA Charles
Extracting video URLs
16
Extracting 

In [36]:
len(battles_all)

100

In [38]:
petes_all[:10]

[2, 5, 7, 6, 6, 8, 6, 4, 6, 2]

In [39]:
battles_all[0].head()

,wins,local,opponent,local_points,opponent_points,duration,date,event,category,round,has_video,url_video
0,1,ABE Hifumi,SERIKZHANOV Yerlan,100,0,02:47,20 Sep 2018,World Championships Baku 2018,-66 kg,Final,True,https://judobase.ijf.org/#/competition/contest...
1,1,ABE Hifumi,AN Baul,10,0,02:38,20 Sep 2018,World Championships Baku 2018,-66 kg,Semi-Final,True,https://judobase.ijf.org/#/competition/contest...
2,1,ABE Hifumi,ZANTARAIA Georgii,100,0,03:10,20 Sep 2018,World Championships Baku 2018,-66 kg,Quarter-Final,True,https://judobase.ijf.org/#/competition/contest...
3,1,ABE Hifumi,VIERU Denis,10,0,00:00,20 Sep 2018,World Championships Baku 2018,-66 kg,Round 4,True,https://judobase.ijf.org/#/competition/contest...
4,1,ABE Hifumi,MEDVES Matteo,100,0,02:57,20 Sep 2018,World Championships Baku 2018,-66 kg,Round 3,True,https://judobase.ijf.org/#/competition/contest...


In [40]:
df = pd.concat(battles_all)

In [43]:
df.shape

(4236, 12)

In [44]:
df.to_csv('data66kg.csv')

### Más atributos

In [194]:
[at for at in dir(value) if 'find' not in at and at[0]!='_']

['clear',
 'click',
 'get_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

## Más funcionalidades

In [117]:
driver.back()

In [ ]:
nameRestaurant.clear()
nameRestaurant.send_keys(nameCity)
nameRestaurant.send_keys(Keys.RETURN)

html_txt = driver.page_source


driver.find_elements_by_xpath("/html/body/div[10]/div[8]/div")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

soup = BeautifulSoup(html_txt, "lxml") 

sec.find("div",class_="popIndex rebrand popIndexDefault").get_text(strip=True) # extraccion